In [19]:
import torch
from torch import nn
from torchvision import models, datasets, transforms
import time
from tqdm.auto import tqdm

In [2]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

In [4]:
num_classes = 10
input_size = 224
batch_size = 128

In [5]:
model = models.resnet18(pretrained=True)
set_requires_grad(model, False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

C:\Users\yaroslav\anaconda3\envs\ml-env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\yaroslav\anaconda3\envs\ml-env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
next(model.fc.parameters()).requires_grad

True

In [7]:
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [8]:
trainset = datasets.CIFAR10(root='./data', train=True,
                            download=True, transform=normalize)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=normalize)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

loaders = {'train': trainloader, 'val': testloader}

# classes = ('plane', 'car', 'bird', 'cat', 'deer',
#            'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cuda', index=0)

In [ ]:
# Optional: проверка выхода AdaptivePooling
# def print_hook(m, i):
#   print("Inside avgpool", i[0].shape)

# handle = model.avgpool.register_forward_pre_hook(print_hook)
# model(torch.ones(1,3,512,512))
# handle.remove()

Inside avgpool torch.Size([1, 512, 16, 16])


In [11]:
model = model.to(device)
pretrain_optimizer = torch.optim.SGD(model.fc.parameters(),
                                     lr=0.001, momentum=0.9)
train_optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
def train_model(model, dataloaders, criterion, optimizer,
                phases, num_epochs=3):
    start_time = time.time()

    acc_history = {k: list() for k in phases}
    loss_history = {k: list() for k in phases}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double()
            epoch_acc /= len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,
                                                       epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                        time_elapsed % 60))

    return model, acc_history

In [15]:
# Сигнатура вызова функции train_model
# train_model(model, loaders, criterion, optimizer,
#             phases=['train', 'val'], num_epochs=num_epochs)

# Pretrain
# запустить предобучение модели на две эпохи
# train_model(model, loaders, criterion, pretrain_optimizer,
#             phases=['train', 'val'], num_epochs=2)

# Train
# запустить дообучение модели
set_requires_grad(model, True)
train_model(model, loaders, criterion, train_optimizer,
            phases=['train', 'val'], num_epochs=2)

Epoch 0/1
----------


  0%|          | 0/391 [00:03<?, ?it/s]

train Loss: 0.3370 Acc: 0.8879


  0%|          | 0/79 [00:02<?, ?it/s]

val Loss: 0.2716 Acc: 0.9073

Epoch 1/1
----------


  0%|          | 0/391 [00:03<?, ?it/s]

train Loss: 0.1776 Acc: 0.9420


  0%|          | 0/79 [00:02<?, ?it/s]

val Loss: 0.1962 Acc: 0.9323

Training complete in 2m 30s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [16]:
num_classes = 1000
set_requires_grad(model, False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

In [35]:
ls

 Том в устройстве C не имеет метки.
 Серийный номер тома: 4ECD-A527

 Содержимое папки C:\Users\yaroslav\Documents\cv-homeworks\finetunning

10.12.2022  20:27    <DIR>          .
10.12.2022  20:27    <DIR>          ..
07.12.2022  20:48    <DIR>          .ipynb_checkpoints
10.12.2022  20:26    <DIR>          data
10.12.2022  20:27            75 908 finetunning-homework.ipynb
               1 файлов         75 908 байт
               4 папок  109 301 235 712 байт свободно


In [36]:
trainset = datasets.ImageNet(root='./data', train=True,
                            download=True, transform=normalize)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.ImageNet(root='./data', train=False,
                           download=True, transform=normalize)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

loaders = {'train': trainloader, 'val': testloader}

RuntimeError: The archive ILSVRC2012_img_train.tar is not present in the root directory or is corrupted. You need to download it externally and place it in ./data.

In [ ]:
train_model(model, loaders, criterion, pretrain_optimizer,
            phases=['train', 'val'], num_epochs=2)